In [41]:
import torch
import numpy as np 
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [42]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [43]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.8818, -2.3398],
        [ 0.3305, -0.2758],
        [-1.1821, -1.7021],
        [-1.8164, -0.4697],
        [ 0.2381,  1.6760],
        [-0.1496,  0.9846],
        [ 0.3982, -1.1078],
        [ 0.4647,  0.6899],
        [ 1.6316,  1.4734],
        [ 0.1720,  0.2715]]) tensor([13.9067,  5.7941,  7.6174,  2.1701, -1.0110,  0.5527,  8.7570,  2.7661,
         2.4532,  3.6138])


In [44]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y


In [45]:

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [47]:
net_sequential = nn.Sequential()
net_sequential.add_module('linear_module', nn.Linear(num_inputs, 1, bias=True))
print(net_sequential)

from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

Sequential(
  (linear_module): Linear(in_features=2, out_features=1, bias=True)
)


In [67]:
for param in net.parameters():
    print(param)



Parameter containing:
tensor([[ 1.9991, -3.3998]], requires_grad=True)
Parameter containing:
tensor([4.1997], requires_grad=True)


In [64]:
loss = nn.MSELoss()

from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍
# print(net[0])

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [65]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        l.backward()
        optimizer.step()
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000059
epoch 2, loss: 0.000083
epoch 3, loss: 0.000147


In [68]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9991, -3.3998]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1997], requires_grad=True)
